## ETF Galaxy

Use this utlity to update the returns and std_dev fields within investment-options.csv

Globals

In [1]:
# Set refresh_timeseries=True to download timeseries.  Otherwise /symbol-cache is used.
refresh_timeseries = True
throttle_limit=100
wait_time=30

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import brownbear as bb

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

In [4]:
# Read in investment options input csv.
df = pd.read_csv('investment-options-in.csv', skip_blank_lines=True, comment='#')
symbols = list(df['Investment Option'])
df

,Investment Option,Description,Asset Class,1 mo,3 mo,1 Yr,3 Yr,5 Yr,Vola,DS Vola,SD 1 Yr,SD 3 Yr,SD 5 Yr
0,SPY,S&P 500 ETF,US Stocks:Large Cap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DIA,DJIA ETF,US Stocks:Large Cap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,QQQ,Nasdaq 100 ETF,US Stocks:Growth,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VTI,Vanguard Total Stock Market Index ETF,US Stocks:Total Stock Market,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IWM,Russell 2000 ETF,US Stocks:Small Cap,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,XOP,S&P Oil & Gas Exploration & Production ETF,US Stocks:Energy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,XHB,S&P Homebuilders ETF,US Stocks:Real Estate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,OIH,Oil Services ETF,US Stocks:Energy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,VDE,Energy ETF,US Stocks:Energy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Eliminate repeat symbols.
symbols = set(list(symbols))

In [6]:
# Get the timeseries for the symbols and compile into a single csv.
bb.fetch_timeseries(symbols, refresh=refresh_timeseries, throttle_limit=throttle_limit, wait_time=wait_time)
bb.compile_timeseries(symbols)

PALL FBTC XLV SPY IBND USO XLK UUP EWJ XLU RSP ETHE GBTC BND VXUS VPL BIL FXI BWZ DBA UGA JPST VCIT DBB IBIT UNG SPYD PPLT EWZ XLY XLRE EWT CORN IWD MDY EBND XLP QQQ EEM BITW XHB FXF PGX AGG XLC SLV NOBL VCSH FXE BWX LQD VNQ VDE PFF IWM EFA XLF FXA IYR IEUR DBE DBP XLE PEY GDX DIA FXC CANE SPSM FXB TLT XLB BNDX XLI EWH QTUM IWF SHY FXY CPER XOP GLD SHV VTI OIH SDY JNK 


In [7]:
# Read symbols timeseries into a dataframe.
df = pd.read_csv('symbols-timeseries.csv', skip_blank_lines=True, comment='#')
df.set_index("Date", inplace=True)
df

,PALL,FBTC,XLV,SPY,IBND,USO,XLK,UUP,EWJ,XLU,...,SHY,FXY,CPER,XOP,GLD,SHV,VTI,OIH,SDY,JNK
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-02,120.03,NaN,76.62,226.95,30.26,78.80,58.40,22.41,45.78,42.70,...,73.56,87.65,16.28,94.58,121.33,94.53,115.74,263.01,75.87,69.96
2019-01-03,122.71,NaN,75.06,221.54,30.27,79.60,55.45,22.29,45.52,42.69,...,73.71,88.82,16.10,93.92,122.43,94.57,113.05,265.21,74.92,69.98
2019-01-04,125.70,NaN,77.30,228.96,30.35,81.44,57.91,22.25,47.06,43.32,...,73.55,88.05,16.52,99.22,121.44,94.59,116.80,278.02,76.66,71.13
2019-01-07,126.76,NaN,77.60,230.76,30.56,82.32,58.42,22.16,47.18,43.03,...,73.50,87.85,16.45,102.62,121.86,94.59,117.97,288.82,76.96,71.92
2019-01-08,128.27,NaN,78.20,232.93,30.30,84.00,58.91,22.23,47.41,43.56,...,73.46,87.80,16.57,103.56,121.53,94.58,119.19,291.75,77.92,72.42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-04-28,86.74,82.77,138.60,550.85,31.39,67.76,208.13,27.22,71.74,78.69,...,82.70,64.88,30.44,114.70,309.07,110.06,271.03,214.76,130.41,94.50
2025-04-29,85.77,83.24,139.28,554.32,31.27,65.85,209.10,27.31,72.02,79.30,...,82.75,64.74,30.18,113.91,306.06,110.06,272.66,215.43,130.99,94.74
2025-04-30,86.00,82.18,140.47,554.54,31.13,63.56,209.97,27.41,71.43,78.90,...,82.86,64.54,28.54,110.71,303.77,110.09,272.82,209.06,131.04,94.18


In [8]:
# Calculate Annual Returns.
annual_returns = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns

PALL    -1.10
FBTC    63.89
XLV      0.31
SPY     14.34
IBND    11.86
        ...  
SHV      4.92
VTI     13.65
OIH    -28.12
SDY      6.08
JNK      8.78
Length: 87, dtype: float64

In [9]:
# Calculate 1 month, 3 months, 1 year, 3 year, and 5 year annualized returns.
annual_returns_1mo = bb.annualized_returns(df, timeperiod='daily', years=1/12)
annual_returns_3mo = bb.annualized_returns(df, timeperiod='daily', years=3/12)
annual_returns_1yr = bb.annualized_returns(df, timeperiod='daily', years=1)
annual_returns_3yr = bb.annualized_returns(df, timeperiod='daily', years=3)
annual_returns_5yr = bb.annualized_returns(df, timeperiod='daily', years=5)

In [10]:
# Calculate 20 day annualized volatility.
daily_returns = df.pct_change(fill_method=None)
years = bb.TRADING_DAYS_PER_MONTH / bb.TRADING_DAYS_PER_YEAR
vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years)

In [11]:
# Calculate 20 day annualized downside volatility.
ds_vola = bb.annualized_standard_deviation(daily_returns, timeperiod='daily', years=years, downside=True)

In [12]:
# Resample df on a monthly basis.
df.index = pd.to_datetime(df.index)
monthly = df.resample('ME').ffill()

In [13]:
# Calculate monthly returns.
monthly_returns = monthly.pct_change(fill_method=None)

In [14]:
# Calculate 1 year, 3 year, and 5 year annualized standard deviation.
std_dev_1yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=1)
std_dev_3yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=3)
std_dev_5yr = bb.annualized_standard_deviation(monthly_returns, timeperiod='monthly', years=5)

In [15]:
# Read investment-options-in.csv
lines = []
with open('investment-options-in.csv', 'r') as f:
    lines = [line.strip() for line in f]

In [16]:
# For each etf, write out the 1 Yr, 3 Yr, 5 Yr, and std dev.
out = []
for line in lines:
    # Copy empty and comment lines.
    if not line or line.startswith('#'):
        out.append(line)
        continue
    # Split line by comma delimiter; strip the fields.
    fields = line.split(',')
    fields = [field.strip() for field in fields]
    symbol = fields[0].strip('\"')
    # Copy header.
    if symbol == 'Investment Option':
        out.append(line)
        continue
    ret_1mo = annual_returns_1mo[symbol]
    ret_3mo = annual_returns_3mo[symbol]
    ret_1yr = annual_returns_1yr[symbol]
    ret_3yr = annual_returns_3yr[symbol]
    ret_5yr = annual_returns_5yr[symbol]
    
    if np.isnan(ret_3yr): ret_3yr = ret_1yr
    if np.isnan(ret_5yr): ret_5yr = ret_3yr

    _vola = vola[symbol]*100
    _ds_vola = ds_vola[symbol]*100
    sd_1yr = std_dev_1yr[symbol]*100
    sd_3yr = std_dev_3yr[symbol]*100
    sd_5yr = std_dev_5yr[symbol]*100

    out.append((
        '{},{},{},"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}",'
        '"{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}","{:0.2f}"'
    ).format(
        fields[0], fields[1], fields[2], ret_1mo, ret_3mo, ret_1yr, ret_3yr,
        ret_5yr, _vola, _ds_vola, sd_1yr, sd_3yr, sd_5yr
    )) 

In [17]:
# Write out asset-classes.csv
with open('investment-options.csv', 'w') as f:
    for line in out:
        f.write(line + '\n')